In [1]:
# 读取数据集，分析2018年的销售数据
import pandas as pd
import chardet
import xgboost
import numpy as np 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import os
import re




def encode_text_dummy(df, name, delete =  False):
    dummies = pd.get_dummies(df[name])
    for x in dummies.columns:
        dummy_name = "{}-{}".format(name, x)
        df[dummy_name] = dummies[x]
    if delete:    
        df.drop(name, axis=1, inplace=True)


In [46]:
def season(i):
    if i in [3,4,5]:
        return "春"
    if i in [6,7,8]:
        return "夏"
    if i in [9,10,11]:
        return "秋"
    if i in [12,1,2]:
        return "冬"
    return np.nan
    

In [163]:
df_.to_csv("KMpre.csv",index=False)

In [164]:
df_=pd.read_csv("KMpre.csv")

In [44]:
#df_["月"] = df_["周"].apply(lambda x: int(x/4.5)+1)

In [47]:
#df_["季节"] = df_["月"].apply(season)

In [165]:
df_.head(3)

,年,周,分公司,中类,销量,销额,-1周销额,-1周销量,-2周销额,-2周销量,...,1周销额,1周销量,2周销额,2周销量,3周销额,3周销量,未来4周销量之和,未来4周销额之和,月,季节
0,2017,42,总部,棉衣,3,210.0,22.0,22,22.0,22,...,6500.0,65,800.0,8,4600.0,46,122,12110.0,10,秋
1,2017,45,总部,棉衣,65,6500.0,210.0,3,22.0,22,...,800.0,8,4600.0,46,24020.0,251,370,35920.0,11,秋
2,2017,46,总部,棉衣,8,800.0,6500.0,65,210.0,3,...,4600.0,46,24020.0,251,6660.0,67,372,36080.0,11,秋


In [59]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split

/home/gang/anaconda3/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [147]:
x = df_[["年","分公司","中类","周","月","季节","-1周销量","-1周销额","-2周销量","-2周销额","-3周销量","-3周销额"]]

In [148]:
x.drop(["周"],axis = 1, inplace =True)

/home/gang/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [149]:
y = df_[['销量', '销额']]#,'1周销额', '1周销量','未来4周销量之和', '未来4周销额之和']]

In [150]:
x.columns

Index(['年', '分公司', '中类', '月', '季节', '-1周销量', '-1周销额', '-2周销量', '-2周销额',
       '-3周销量', '-3周销额'],
      dtype='object')

In [151]:
encode_text_dummy(x,name = "分公司",delete = True)

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/home/gang/anaconda3/lib/python3.6/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [152]:
encode_text_dummy(x,name = "中类",delete = True)

/home/gang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [153]:
encode_text_dummy(x,name = "季节",delete = True)

In [155]:
encode_text_dummy(x,name = "月",delete = True)

In [156]:
encode_text_dummy(x,name = "年",delete = True)

In [166]:
x.head(3)

,-1周销量,-1周销额,-2周销量,-2周销额,-3周销量,-3周销额,分公司-华中1分公司,分公司-华中2分公司,分公司-华北分公司,分公司-华南分公司,...,月-8,月-9,月-10,月-11,月-12,年-2015,年-2016,年-2017,年-2018,年-2019
0,22,22.0,22,22.0,22,22.0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
1,3,210.0,22,22.0,22,22.0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0
2,65,6500.0,3,210.0,22,22.0,0,0,0,0,...,0,0,0,1,0,0,0,1,0,0


In [169]:
y.head(3)

,销量,销额
0,3,210.0
1,65,6500.0
2,8,800.0


In [177]:
train_x,test_x,train_y,test_y = train_test_split(x,y,train_size = 0.8)

/home/gang/anaconda3/lib/python3.6/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


In [179]:
regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=30,max_depth=3,max_features="sqrt", random_state=24))
regr_multirf.fit(train_x, train_y)
pred_y = regr_multirf.predict(test_x)
y_test=test_y.values
            
for i in range(2):
    error = abs(pred_y[:,i]-y_test[:,i])
    scaled = (error/pred_y[:,i])*100
    score = scaled.mean()
    print(score) #舍弃周次的encode

59.462759156103786
61.344729223137854


In [183]:
for i_ in [30,50,100,130,170,200]:
    for j_ in [3,7,10,15,20,30,50]:
        for k_ in ["sqrt","log2"]:
            
            regr_multirf = MultiOutputRegressor(RandomForestRegressor(n_estimators=i_,max_depth=j_,max_features=k_, random_state=24))
            regr_multirf.fit(train_x, train_y)
            pred_y = regr_multirf.predict(test_x)
            y_test=test_y.values
            print(" ")
            print(i_,j_,k_)
            
            for i in range(2):
                error = abs(pred_y[:,i]-y_test[:,i])
                scaled = (error/pred_y[:,i])*100
                score = scaled.mean()
                print(score) #舍弃周次的encode
            
            

 
30 3 sqrt
59.462759156103786
61.344729223137854
 
30 3 log2
64.41005901270876
65.18259833296155
 
30 7 sqrt
47.66438389799265
48.83957233092827
 
30 7 log2
52.37496418628498
52.35372794319635
 
30 10 sqrt
43.98809514799893
42.6798253946022
 
30 10 log2
46.61871120493663
46.1314924495848
 
30 15 sqrt
38.64540290141184
37.356802310555246
 
30 15 log2
40.56438240186026
39.425194690707485
 
30 20 sqrt
37.32305365667184
35.525337972844795
 
30 20 log2
37.55832656847055
36.47842486645883
 
30 30 sqrt
36.71781554163758
35.15917901987722
 
30 30 log2
36.57770247643078
35.55612150877084
 
30 50 sqrt
36.79228540520424
35.96574121672086
 
30 50 log2
35.76098007838725
35.45557912576606
 
50 3 sqrt
58.71564954493816
60.02261426269404
 
50 3 log2
63.31598251100884
64.07688123409179
 
50 7 sqrt
47.587565815883984
48.97919639258536
 
50 7 log2
51.887590007027086
51.95166375298732
 
50 10 sqrt
43.68559880325944
42.80805527837369
 
50 10 log2
46.498143467394684
46.17132973963625
 
50 15 sqrt
38.644835

In [172]:
for i in [30,50,100,130,170,200]:
    for j in [3,7,10,15,20]:
        for k in ["sqrt","log2"]:
            print(i,j,k)

30 3 sqrt
30 3 log2
30 7 sqrt
30 7 log2
30 10 sqrt
30 10 log2
30 15 sqrt
30 15 log2
30 20 sqrt
30 20 log2
50 3 sqrt
50 3 log2
50 7 sqrt
50 7 log2
50 10 sqrt
50 10 log2
50 15 sqrt
50 15 log2
50 20 sqrt
50 20 log2
100 3 sqrt
100 3 log2
100 7 sqrt
100 7 log2
100 10 sqrt
100 10 log2
100 15 sqrt
100 15 log2
100 20 sqrt
100 20 log2
130 3 sqrt
130 3 log2
130 7 sqrt
130 7 log2
130 10 sqrt
130 10 log2
130 15 sqrt
130 15 log2
130 20 sqrt
130 20 log2
170 3 sqrt
170 3 log2
170 7 sqrt
170 7 log2
170 10 sqrt
170 10 log2
170 15 sqrt
170 15 log2
170 20 sqrt
170 20 log2
200 3 sqrt
200 3 log2
200 7 sqrt
200 7 log2
200 10 sqrt
200 10 log2
200 15 sqrt
200 15 log2
200 20 sqrt
200 20 log2


In [158]:
regr_multirf = MultiOutputRegressor(RandomForestRegressor(max_depth=10, random_state=24))
regr_multirf.fit(train_x, train_y)

MultiOutputRegressor(estimator=RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=24, verbose=0, warm_start=False),
           n_jobs=1)

In [159]:
pred_y = regr_multirf.predict(test_x)

In [62]:
pred_y.shape

(3538, 6)

In [246]:
pred_y[:,1].shape

(3538,)

In [64]:
test_y.values.shape

(3538, 6)

In [160]:
y_test=test_y.values

In [162]:
for i in range(2):
    error = abs(pred_y[:,i]-y_test[:,i])
    scaled = (error/pred_y[:,i])*100
    score = scaled.mean()
    print(score) #舍弃周次的encode

41.250152094073925
39.742880210928256


In [146]:
for i in range(6):
    error = abs(pred_y[:,i]-y_test[:,i])
    scaled = (error/pred_y[:,i])*100
    score = scaled.mean()
    print(score) #舍弃周次的encode

44.18706478154811
41.54163583949868
52.55603885219698
53.19185571894473
48.13798812374883
49.53754519183458


In [115]:
for i in range(6):
    error = abs(pred_y[:,i]-y_test[:,i])
    scaled = (error/pred_y[:,i])*100
    score = scaled.mean()
    print(score)#舍弃周次和月

40.97488456099869
40.40080832433757
52.34153178700889
51.43323895920127
47.95297974925523
50.76464563759192


In [ ]:
for i in range(6):
    error = abs(pred_y[:,i]-y_test[:,i])
    scaled = (error/pred_y[:,i])*100
    score = scaled.mean()
    print(score)

In [132]:
for i in range(6):
    error = abs(pred_y[:,i]-y_test[:,i])
    scaled = (error/pred_y[:,i])*100
    score = scaled.mean()
    print(score) #全部encode

38.95104063619484
40.035018685666415
54.30621491568478
56.901934483677046
50.65533289996525
52.13003839787711


误差

In [294]:
(pred_y==0).sum()

0

In [283]:
y_test = test_y.values

In [282]:
pred_y.dtype

dtype('float64')

In [286]:
pred_y=pred_y.astype("float64")
y_test=y_test.astype("float64")